# Pre-processing News Data

Danh sách sinh viên đi học:
1. Đinh Ngọc Khánh Huyền 20225726
2. Trần Khánh Quỳnh 20225762
3. Bùi Ý Nhi 20225657

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Sử dụng phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

# Các bước làm

## Chuẩn bị các thư viện cần thiết

In [2]:
!pip install pyvi

  Using cached pyvi-0.1.1-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached sklearn_crfsuite-0.5.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached python_crfsuite-0.9.11-cp313-cp313-win_amd64.whl.metadata (4.4 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached pyvi-0.1.1-py2.py3-none-any.whl (8.5 MB)

   ---------------------------------------- 0/5 [tqdm]
   ---------------------------------------- 0/5 [tqdm]
   ---------------------------------------- 0/5 [tqdm]
   ---------------------------------------- 0/5 [tqdm]
   -------- ------------------------------- 1/5 [tabulate]
   ---------------- ----------------------- 2/5 [python-crfsuite]
   ------------------------ --------------- 3/5 [sklearn-crfsuite]
   -------------------------------- ------- 4/5 [pyvi]
   -------------------------------- ------- 4/5 [pyvi]
   ---------------------------------------- 5/5 [pyvi]



In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau

- data/news_vnexpress/

    - Kinh tế:
        - bài báo 1.txt
        - bài báo 2.txt
    - Pháp luật
        - bài báo 3.txt
        - bài báo 4.txt

In [12]:
#Có thể chỉnh lại đường dẫn thư mục cho phù hợp
INPUT = 'news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [13]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    # Bỏ qua file desktop.ini và chỉ xử lý thư mục
    if os.path.isdir(os.path.join(INPUT, label)) and label != 'desktop.ini':
        print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
        n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 121
du-lich: 55
giai-tri: 202
giao-duc: 106
khoa-hoc: 145
kinh-doanh: 263
phap-luat: 60
suc-khoe: 163
the-thao: 174
thoi-su: 60
-------------------------
Tổng số văn bản: 1349


In [27]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['news_vnexpress\\kinh-doanh\\00212.txt']
[5]
['Đại diện Công ty TNHH Bảo hiểm Nhân thọ Sun Life Việt Nam (Sun Life Việt Nam) chi biết, khi nhận được thông tin ông Tống Văn Đại bị bệnh hiểm nghèo, đơn vị đã cắt cử nhân viên đến thăm hỏi sức khỏe khách hàng.Sun Life Việt Nam cũng tiến hành các thủ tục để chi trả quyền lợi như đã cam kết của hợp đồng bảo hiểm. Với số tiền được chi trả từ quyền lợi bảo hiểm, Sun Life Việt Nam mong muốn ông Đại và người thân có thêm nguồn tài chính để yên tâm điều trị bệnh, bồi dưỡng sức khỏe, mau chóng trở lại với một cuộc sống khỏe mạnh hơn.Tại lễ chi trả quyền lợi bảo hiểm vào 21/2, khách hàng còn được tư vấn, giới thiệu về các giải pháp tài chính bảo hiểm của Sun Life Việt Nam giúp người dùng được bảo vệ tài chính trước rủi ro và mang đến cơ hội gia tăng tài sản. Khách hàng có cơ hội hiểu

## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [28]:
# load dữ liệu các stopwords
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    vietnamese_stopwords = f.read().splitlines()
#---> Code ở đây
def preprocess_text(text):
    # Tiền xử lý văn bản: chuyển về chữ thường, loại bỏ ký tự đặc biệt, số, và tokenization
    text = text.lower()  # Chuyển về chữ thường
    text = ViTokenizer.tokenize(text)  # Tokenization
    # Loại bỏ ký tự đặc biệt và số
    text = ''.join(char for char in text if char.isalpha() or char.isspace())
    return text

txt = preprocess_text(data_train.data[0])
print(f"\nVăn bản sau khi tiền xử lý:\n{txt}")



Văn bản sau khi tiền xử lý:
đạidiện côngtytnhh bảohiểmnhânthọ sun life việtnam  sun life việtnam  chi biết  khi nhận được thôngtin ông tống văn đại bị bệnh hiểmnghèo  đơnvị đã cắtcửnhânviên đến thămhỏi sứckhỏe kháchhàng  sun life việtnam cũng tiếnhành các thủtục để chitrả quyềnlợi như đã camkết của hợpđồng bảohiểm  với số tiền được chitrả từ quyềnlợi bảohiểm  sun life việtnam mongmuốn ông đại và ngườithân có thêm nguồn tàichính để yêntâm điềutrị bệnh  bồidưỡngsứckhỏe  mauchóng trởlại với một cuộcsống khỏe mạnh hơn  tại lễ chitrả quyềnlợi bảohiểm vào     kháchhàng còn được tưvấn  giớithiệu về các giảipháp tàichính bảohiểm của sun life việt nam giúp người dùng được bảovệ tàichính trước rủiro và mang đến cơhội giatăng tàisản  kháchhàng có cơhội hiểu thêm về các sảnphẩm bảohiểm bổsung  bảovệ sứckhỏe toàndiện  từ       sun life việtnam đã chitrả quyềnlợi bảohiểm cho hơn  lượt kháchhàng với tổngsố tiền hơn  tỷ đồng  việc chitrả quyềnlợi bảohiểm đúng  đủ và nhanhchóng là một trong những ưuti

In [35]:
# Chuyển hoá dữ liệu text về dạng vector TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo vectorizer với stopwords tiếng Việt và giới hạn từ điển
vectorizer = TfidfVectorizer(
    stop_words=vietnamese_stopwords,
    max_features=10000,  # Giới hạn số từ trong từ điển là 5000 từ phổ biến nhất
    ngram_range=(1, 2)  # Sử dụng unigram và bigram
)

# Tiền xử lý tất cả văn bản và chuyển đổi thành ma trận TF-IDF
preprocessed_texts = [preprocess_text(text) for text in data_train.data]
X = vectorizer.fit_transform(preprocessed_texts)
Y = data_train.target

# Biến lưu từ điển
module_count_vector = vectorizer  # Biến này lưu vectorizer (chứa từ điển)
vocabulary = vectorizer.vocabulary_  # Từ điển dạng {từ: index}
feature_names = vectorizer.get_feature_names_out()  # Danh sách các từ trong từ điển

print(f"\nSố lượng từ trong từ điển: {len(vocabulary)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")
print(f"Một số từ đầu tiên trong từ điển: {list(feature_names[:10])}")


Số lượng từ trong từ điển: 10000
Kích thước dữ liệu sau khi xử lý: (1349, 10000)
Kích thước nhãn tương ứng: (1349,)
Một số từ đầu tiên trong từ điển: ['about', 'ac', 'ac milan', 'acb', 'ace', 'acid', 'acrylamide', 'adn', 'advanced', 'afc']


In [34]:
# Hiển thị một số thông tin về dữ liệu đã xử lý
print("Các nhãn (categories):")
for i, category in enumerate(data_train.target_names):
    print(f"{i}: {category}")

print(f"\nVí dụ văn bản đầu tiên (sau khi vector hóa):")
print(f"Nhãn: {Y[0]} ({data_train.target_names[Y[0]]})")
print(f"Số từ khác 0 trong vector: {X[0].nnz}")

# Hiển thị một số từ có trọng số cao nhất (sử dụng feature_names đã định nghĩa trong cell trước)
first_doc_vector = X[0].toarray()[0]
top_indices = first_doc_vector.argsort()[-10:][::-1]
print(f"\n10 từ có trọng số TF-IDF cao nhất trong văn bản đầu tiên:")
for idx in top_indices:
    if first_doc_vector[idx] > 0:
        print(f"  {feature_names[idx]}: {first_doc_vector[idx]:.4f}")

Các nhãn (categories):
0: doi-song
1: du-lich
2: giai-tri
3: giao-duc
4: khoa-hoc
5: kinh-doanh
6: phap-luat
7: suc-khoe
8: the-thao
9: thoi-su

Ví dụ văn bản đầu tiên (sau khi vector hóa):
Nhãn: 5 (kinh-doanh)
Số từ khác 0 trong vector: 112

10 từ có trọng số TF-IDF cao nhất trong văn bản đầu tiên:
  life: 0.5370
  sun: 0.4888
  kháchhàng: 0.2799
  bảohiểm: 0.2722
  quyềnlợi: 0.2407
  chitrả: 0.2240
  việtnam: 0.1730
  tàichính: 0.1289
  việt: 0.0877
  nam: 0.0760


In [36]:
print(X[100]) #Sau khi xử lí, dữ liệu được lưu dưới dạng ma trận thưa như sau:

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 128 stored elements and shape (1, 10000)>
  Coords	Values
  (0, 3527)	0.062497157994849464
  (0, 7218)	0.0753700653539765
  (0, 2433)	0.03485262716947387
  (0, 4065)	0.02489625393475355
  (0, 1310)	0.023095256441280073
  (0, 4258)	0.04963906732451894
  (0, 3517)	0.07634915914061835
  (0, 4295)	0.03329315918712316
  (0, 3680)	0.02845492965587408
  (0, 4129)	0.0277897223722982
  (0, 9892)	0.025959667295790054
  (0, 8781)	0.028268391704660223
  (0, 3485)	0.11452373871092753
  (0, 2312)	0.055119885132005386
  (0, 6686)	0.02076244331266875
  (0, 9080)	0.08267982769800808
  (0, 9212)	0.050057346103789285
  (0, 8599)	0.045208329111718536
  (0, 1662)	0.034985052343614584
  (0, 8254)	0.07796403230098778
  (0, 4498)	0.04546068835338704
  (0, 8274)	0.023837653491859807
  (0, 6036)	0.08389842687987194
  (0, 6487)	0.06007265714772476
  (0, 2974)	0.27992717613267193
  :	:
  (0, 844)	0.043996078688156186
  (0, 3175)	0.050393776201755974
  